In [72]:
import PyPDF2
import re
import pandas as pd
import nltk
import os
import sklearn
import requests
from bs4 import BeautifulSoup
import csv

In [87]:
pdf_text = []
date = []

In [88]:
pdf_list = {}
for dire in os.listdir("pdfs"):
    temp = []
    temp.append(os.listdir("pdfs/" + dire))
    pdf_list[dire] = temp

In [89]:
csv_list = []
for dire in os.listdir("converted pdfs/"):
    csv_list.append(dire)

In [90]:
pdf_list

{'1954': [['histmin19540303.pdf',
   'histmin19540623.pdf',
   'histmin19540922.pdf',
   'histmin19541207.pdf']],
 '1955': [['histmin19550111.pdf',
   'histmin19550302.pdf',
   'histmin19550510.pdf',
   'histmin19550622.pdf',
   'histmin19550712.pdf',
   'histmin19550802.pdf',
   'histmin19550823.pdf',
   'histmin19550914.pdf',
   'histmin19550926.pdf',
   'histmin19551004.pdf',
   'histmin19551025.pdf',
   'histmin19551116.pdf',
   'histmin19551130.pdf',
   'histmin19551208.pdf',
   'histmin19551213.pdf']],
 '1956': [['histmin19560110.pdf',
   'histmin19560124.pdf',
   'histmin19560215.pdf',
   'histmin19560306.pdf',
   'histmin19560327.pdf',
   'histmin19560417.pdf',
   'histmin19560509.pdf',
   'histmin19560523.pdf',
   'histmin19560605.pdf',
   'histmin19560626.pdf',
   'histmin19560717.pdf',
   'histmin19560807.pdf',
   'histmin19560821.pdf',
   'histmin19560911.pdf',
   'histmin19560925.pdf',
   'histmin19561016.pdf',
   'histmin19561113.pdf',
   'histmin19561127.pdf',
   'histmi

In [91]:
csv_list

['tabula-histmin19620529.csv',
 'tabula-histmin19620619.csv',
 'tabula-histmin19621002.csv',
 'tabula-histmin19621113.csv',
 'tabula-histmin19630212.csv',
 'tabula-histmin19640303.csv',
 'tabula-histmin19640324.csv',
 'tabula-histmin19640617.csv',
 'tabula-histmin19641124.csv']

In [78]:
def csv_import(file):
    text = ""
    with open("converted pdfs/" + file) as csv_file:
        temp = csv.reader(csv_file)
        for row in temp:
            for item in row:
                if item != "":
                    text += item + " "
    return(text)

In [92]:
for i in csv_list:
    date.append(i[-12:-4])
    pdf_text.append(csv_import(i))

In [80]:
pdf_text

['A meeting of the Federal Open Market Comittee was held in the offices of the Board of Governors of the Federal Reserve System in Washington on Tuesday, May 29, 1962, at 10:00 a.m. PRESENT: Mr.  Martin, Chairman 1/ Mr.  Hayes, Vice Chairman Mr.  Balderston Mr.  Bryan Mr.  Deming Mr.  Ellis Mr.  Fulton Mr.  King Mr.  Mills Mr.  Mitchell Mr.  Robertson Mr. Shepardson Messrs. Bopp, Scanlon, Clay, and Irons, Alternate Members of the Federal Open Market Committee Messrs. Wayne and Swan, Presidents of the Federal Reserve Banks of Richmond and San Francisco, respectively Mr. Sherman, Assistant Secretary Mr. Kenyon, Assistant Secretary Mr. Hackley, General Counsel Mr. Noyes, Economist Messrs. Brandt, Brill, Furth, Garvy, Holland, Koch, and Willis, Associate Economists Mr. Stone, Manager, System Open Market Account Mr. Coombs, Special Manager, System Open Market Account Mr. Molony, Assistant to the Board of Governors Mr. Cardon, Legislative Counsel, Board of Governors Mr. Williams, Adviser, Di

In [ ]:
csv_import(csv_list[0])

In [ ]:
text = ""
with open("converted pdfs/" + csv_list[0]) as csv_file:
    temp = csv.reader(csv_file)
    for row in temp:
        for item in row:
            if item != "":
                text += item + " "
print(text)

In [81]:
url = "https://www.federalreserve.gov/monetarypolicy/fomchistorical{}.htm"
base_url = "https://www.federalreserve.gov"
def get_html_min(year):
    r = requests.get(url.format(year))
    soup = BeautifulSoup(r.text, "html.parser")
    links = soup.find_all("a")
    minutes = []
    
    for link in links:
        if "Minutes" in link.text:
            minutes.append(link)
            
    minutes_links = []
    
    for link in minutes:
        minutes_links.append(requests.compat.urljoin(base_url, link.get('href')))
    
    for meeting in minutes_links:
        if re.findall("\#", meeting) == ["#"]:
            break
        elif meeting[-7:-4] == "min":
            date.append(meeting[-15:-7])
        else:
            date.append(meeting[-12:-4])
        print(meeting)
        r = requests.get(meeting)
        soup = BeautifulSoup(r.text, "html.parser")
        min_text = soup.body.get_text()
        min_text = re.sub("\n", " ", min_text)
        min_text = re.sub("\r", " ", min_text)
        min_text = re.sub("\x92", "'", min_text)
        min_text = re.sub("\x97")
#        minutes_text = nltk.tokenize.word_tokenize(min_text)
        pdf_text.append(min_text)

In [93]:
for key in range(1954, 1993):
    for values in pdf_list[str(key)]:
        for pdf in values:
            print(pdf)
            date.append(pdf[-12:-4])
            with open("C:/Users/mjcor/Documents/GitHub/fed_reserve_text_project/pdfs/" + str(key) + "/" + pdf, "rb") as pdfFileObjs:
                pdfReader = PyPDF2.PdfFileReader(pdfFileObjs)
                temp = ""
                for i in range(pdfReader.numPages):
                    pageObj = pdfReader.getPage(i)
                    text = pageObj.extractText()
                    text = re.sub("Š", "-", text)
                    text = re.sub("™", "'", text)
                    text = re.sub("\n", "", text)
#                    text = nltk.tokenize.word_tokenize(text)
                    temp = temp + " " + text
                pdf_text.append(temp)

histmin19540303.pdf


histmin19540623.pdf
histmin19540922.pdf
histmin19541207.pdf
histmin19550111.pdf
histmin19550302.pdf
histmin19550510.pdf
histmin19550622.pdf
histmin19550712.pdf
histmin19550802.pdf
histmin19550823.pdf
histmin19550914.pdf
histmin19550926.pdf
histmin19551004.pdf
histmin19551025.pdf
histmin19551116.pdf
histmin19551130.pdf
histmin19551208.pdf
histmin19551213.pdf
histmin19560110.pdf
histmin19560124.pdf
histmin19560215.pdf
histmin19560306.pdf
histmin19560327.pdf
histmin19560417.pdf
histmin19560509.pdf
histmin19560523.pdf
histmin19560605.pdf
histmin19560626.pdf
histmin19560717.pdf
histmin19560807.pdf
histmin19560821.pdf
histmin19560911.pdf
histmin19560925.pdf
histmin19561016.pdf
histmin19561113.pdf
histmin19561127.pdf
histmin19561210.pdf
histmin19570108.pdf
histmin19570128.pdf
histmin19570218.pdf
histmin19570305.pdf
histmin19570326.pdf
histmin19570416.pdf
histmin19570424.pdf
histmin19570507.pdf
histmin19570528.pdf
histmin19570618.pdf
histmin19570709.pdf
histmin19570730.pdf
histmin19570820.pdf


fomcmoa19810331.pdf
fomcmoa19810506.pdf
fomcmoa19810518.pdf
fomcmoa19810707.pdf
fomcmoa19810807.pdf
fomcmoa19810818.pdf
fomcmoa19811006.pdf
fomcmoa19811117.pdf
fomcmoa19811222.pdf
fomcmoa19820202.pdf
fomcmoa19820330.pdf
fomcmoa19820518.pdf
fomcmoa19820701.pdf
fomcmoa19820715.pdf
fomcmoa19820824.pdf
fomcmoa19821005.pdf
fomcmoa19821116.pdf
fomcmoa19821221.pdf
fomcmoa19830209.pdf
fomcmoa19830329.pdf
fomcmoa19830524.pdf
fomcmoa19830713.pdf
fomcmoa19830823.pdf
fomcmoa19831004.pdf
fomcmoa19831115.pdf
fomcmoa19831220.pdf
fomcmoa19840131.pdf
fomcmoa19840327.pdf
fomcmoa19840522.pdf
fomcmoa19840717.pdf
fomcmoa19840821.pdf
fomcmoa19841002.pdf
fomcmoa19841107.pdf
fomcmoa19841218.pdf
fomcmoa19850213.pdf
fomcmoa19850326.pdf
fomcmoa19850521.pdf
fomcmoa19850710.pdf
fomcmoa19850820.pdf
fomcmoa19851001.pdf
fomcmoa19851105.pdf
fomcmoa19851217.pdf
fomcmoa19860212.pdf
fomcmoa19860401.pdf
fomcmoa19860520.pdf
fomcmoa19860709.pdf
fomcmoa19860819.pdf
fomcmoa19860923.pdf
fomcmoa19861105.pdf
fomcmoa19861216.pdf


In [94]:
for year in range(1993, 2008):
    get_html_min(year)

https://www.federalreserve.gov/fomc/MINUTES/1993/19930203min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19930323min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19930707min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19930817min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19930921min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19931116min.htm
https://www.federalreserve.gov/fomc/MINUTES/1993/19931221min.htm
https://www.federalreserve.gov/fomc/MINUTES/1994/19940204min.htm
https://www.federalreserve.gov/fomc/MINUTES/1994/19940322min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950201min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950328min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950523min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950706min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950822min.htm
https://www.federalreserve.gov/fomc/MINUTES/1995/19950926min.htm
https://www.federalreserv

In [105]:
for key in range(2008, 2019):
    for values in pdf_list[str(key)]:
        for pdf in values:
            if pdf[-12:-4] == "confcall":
                dte = "2" + pdf[-20:-12]
            elif pdf[-11:-4] == "meeting":
                dte = pdf[-19:-11]
            else:
                dte = pdf[-12:-4]
            print(dte)
            date.append(dte)
            with open("C:/Users/mjcor/Documents/GitHub/fed_reserve_text_project/pdfs/" + str(key) + "/" + pdf, "rb") as pdfFileObjs:
                pdfReader = PyPDF2.PdfFileReader(pdfFileObjs)
                temp = ""
                for i in range(pdfReader.numPages):
                    pageObj = pdfReader.getPage(i)
                    text = pageObj.extractText()
                    text = re.sub("Š", "-", text)
                    text = re.sub("™", "'", text)
                    text = re.sub("\n", "", text)
#                    text = nltk.tokenize.word_tokenize(text)
                    temp = temp + " " + text
                pdf_text.append(temp)

20080109
20080121
20080310
20080724
20080929
20081007
20080130
20080318
20080430
20080625
20080805
20080916
20081029
20081216
20090116
20090207
20090603
20090128
20090318
20090429
20090624
20090812
20090923
20091104
20091216
20100509
20101015
20100127
20100316
20100428
20100623
20100810
20100921
20101103
20101214
20110801
20111128
20110126
20110315
20110427
20110622
20110809
20110921
20111102
20111213
20120125
20120313
20120425
20120620
20120801
20120913
20121024
20121212
20130130
20130320
20130501
20130619
20130731
20130918
20131030
20131218
20140129
20140319
20140430
20140618
20140730
20140917
20141029
20141217
20150128
20150318
20150429
20150617
20150729
20150917
20151028
20151216
20160127
20160316
20160427
20160615
20160727
20160921
20161102
20161214
20170201
20170315
20170503
20170614
20170726
20170920
20171101
20171213
20180131
20180321


In [106]:
print(len(pdf_text), len(date))

710 710


In [107]:
text_df = pd.DataFrame({"Date": date,
                        "Text": pdf_text})

In [108]:
text_df

,Date,Text
0,19620529,A meeting of the Federal Open Market Comittee ...
1,19620619,A meeting of the Federal Open Market Comittee ...
2,19621002,A meeting of the Federal Open Market Committee...
3,19621113,A meeting of the Federal Open Market Committee...
4,19630212,A meeting of the Federal Open Market Committee...
5,19640303,A meeting of the Federal Open Market Committee...
6,19640324,A meeting of the Federal Open Market Committee...
7,19640617,A meeting of the Federal Open Market. Committe...
8,19641124,A meeting of the Federal Open Market Committee...
9,19540303,A meeting of the Federal Open Market Committe...


In [136]:
?re.sub

In [151]:
text_df.iloc[615]["Text"]

" Conference Call of the Federal Open Market Committee on January 9, 2008  A conference call of the Federal Open Market Committee was held on Wednesday, January 9, 2008, at 5:00 p.m.  Those present were the following:  Mr. Bernanke, Chairman Mr. Geithner, Vice Chairman Mr. Evans Mr. Hoenig Mr. Kohn Mr. Kroszner Mr. Mishkin Mr. Poole Mr. Rosengren Mr. Warsh  Ms. Cumming, Mr. Fisher, Ms. Pianalto, and Messrs. Plosser and Stern, Alternate Members of the Federal Open Market Committee  Messrs. Lacker and Lockhart, and Ms. Yellen, Presidents of the Federal Reserve Banks of Richmond, Atlanta, and San Francisco, respectively  Mr. Madigan, Secretary and Economist Ms. Danker, Deputy Secretary Ms. Smith, Assistant Secretary Mr. Skidmore, Assistant Secretary Mr. Alvarez, General Counsel Mr. Baxter, Deputy General Counsel Mr. Sheets, Economist Mr. Stockton, Economist  Messrs. Connors, Fuhrer, Kamin, Rasche, Sellon, Slifman, Tracy, and Wilcox, Associate Economists  Mr. Dudley, Manager, System Open M

In [156]:
for i in range(0, 710):
    with open("texts/" + text_df.iloc[i]["Date"] + ".txt", "w") as text_file:
        temp_string = re.sub("\x92", "'", text_df.iloc[i]["Text"])
        temp_string = re.sub("\x97", " ", temp_string)
        temp_string = re.sub("\x93", " ", temp_string)
        temp_string = re.sub("\x94", " ", temp_string)
        temp_string = re.sub("\ufb01", " ", temp_string)
        temp_string = re.sub("\ufb02", " ", temp_string)
        temp_string = re.sub("\u0131", " ", temp_string)
        text_file.write(temp_string)

IndexError: single positional indexer is out-of-bounds

In [133]:
text_df.iloc[709]["Text"]

'     Minutes of the Federal Open Market Committee March 20Œ21, 2018  A joint meeting of the Federal Open Market Committee and the Board of Governors was held in the offices of the Board of Governors of the Federal Reserve System in Washington, D.C., on Tuesday, March 20, 2018, at 1:00 p.m. and continued on Wednesday, March 21, 2018, at 9:00 a.m.1 PRESENT: Jerome H. Powell, Chairman William C. Dudley, Vice Chairman Thomas I. Barkin Raphael W. Bostic Lael Brainard Loretta J. Mester Randal K. Quarles John C. Williams  James Bullard, Charles L. Evans, Esther L. George, Eric Rosengren, and Michael Strine,2 Alternate Members of the Federal Open Market Committee  Patrick Harker, Robert S. Kaplan, and Neel Kashkari, Presidents of the Federal Reserve Banks of Philadelphia, Dallas, and Minneapolis, respectively  James A. Clouse, Secretary Matthew M. Luecke, Deputy Secretary David W. Skidmore, Assistant Secretary Michelle A. Smith, Assistant Secretary Mark E. Van Der Weide, General Counsel Micha

In [ ]:
print(date)

In [ ]:
with open("C:/Users/mjcor/Documents/GitHub/fed_reserve_text_project/pdfs/1964/histmin19641124.pdf", "rb") as pdfFileObjs:
    pdfReader = PyPDF2.PdfFileReader(pdfFileObjs)
    temp = []
    for i in range(pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        print(i, pageObj)
        text = pageObj.extractText()
        text = re.sub("Š", "-", text)
        text = re.sub("™", "'", text)
        text = re.sub("\n", "", text)
        text = nltk.tokenize.word_tokenize(text)
        temp.append(text)
    flattext = [item.lower() for page in temp for item in page]

print(flattext)

In [ ]:
total_len = 0
for i in range(0, len(pdf_text)):
    total_len += len(pdf_text[i])
    
print(total_len)

In [ ]:
for i in os.listdir("converted pdfs/"):
    

In [ ]:
for i in range(0, len(pdf_text)):
    if len(pdf_text[i]) == 0:
        print(date[i])

In [ ]:
for i in range(0, len(pdf_text)):
    
    fd = nltk.FreqDist(pdf_text[i])
    print(date[i], fd.most_common(1))

In [ ]:
fd = nltk.FreqDist(pdf_text[0])

In [ ]:
stop = [word for word in nltk.corpus.stopwords.words()]
stop.append(".")
stop.append(",")
stop.append("mr.")
for i in range(0, len(pdf_text)):
    item = pdf_text[i]
    filtered_words = [word.lower() for word in item if word.lower() not in stop]
    fd = nltk.FreqDist(filtered_words)
    print(date[i], fd.most_common(5))

In [ ]:
pdfFileObj = open("C:/Users/mjcor/Desktop/fomcmoa19780117.pdf", "rb")

In [ ]:
pdfFileObj = open("C:/Users/mjcor/Desktop/FOMC20050202meeting.pdf", "rb")

In [ ]:
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

In [ ]:
pdfReader.numPages

In [ ]:
pageObj = pdfReader.getPage(0)

In [ ]:
pageObj.extractText()

In [ ]:
fomc1978 = []
for i in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    text = pageObj.extractText()
    text = re.sub("Š", "-", text)
    text = re.sub("™", "'", text)
    text = re.sub("\n", "", text)
    text = nltk.tokenize.word_tokenize(text)
    fomc1978.append(text)

In [ ]:
fomc1978

In [ ]:
FOMC20050202meeting

In [ ]:
flattext = [item.lower() for page in fomc1978 for item in page]
flattext

In [ ]:
tri = list(nltk.trigrams(flattext))
tri_fd = nltk.FreqDist(tri)
tri_fd.most_common(30)

In [ ]:
big = list(nltk.bigrams(flattext))
big_fd = nltk.FreqDist(big)
big_fd.most_common(30)

In [ ]:
FOMC_FD = nltk.FreqDist(flattext)

In [ ]:
FOMC_FD.most_common(30)


In [ ]:
nltk.corpus.stopwords.words()

In [ ]:
stop = [word for word in nltk.corpus.stopwords.words()]
filtered_words = [word for word in flattext if word not in stop]

In [ ]:
FOMC_FD = nltk.FreqDist(filtered_words)

In [ ]:
FOMC_FD.most_common(30)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
string = ""
for item in pdf_text[0]:
    string = string + " " + item
    
print(string)

In [ ]:
sparse_text = []
for text in pdf_text:
    temp = ""
    for item in text:
        temp = temp + " " + item
    sparse_text.append(temp)

In [ ]:
X = vectorizer.fit_transform(sparse_text)

In [ ]:
X

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf = False)
transformer

In [ ]:
?TfidfTransformer

In [ ]:
?transformer.fit_transform

In [ ]:
tfidf = transformer.fit_transform(X)

In [ ]:
transformer.fit_transform(X).toarray()

In [ ]:
tfidf.toarray()

In [ ]:
vectorizer.get_feature_names()[530]

In [ ]:
trans.toarray()

In [ ]:
trans.shape

In [ ]:
tfidf.shape

In [ ]:
target = max(tfidf.toarray()[0])
for i, x in enumerate(tfidf.toarray()[0]):
    if x == target:
        print(i, x)

In [ ]:
len(tfidf.toarray()[2])

In [ ]:
for i in range(0, 701):
    print(i, tfidf[i,51391])

In [ ]:
transformer.idf_[51391]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words="english")
tfidf = vectorizer.fit_transform(sparse_text)

In [ ]:
simple_text = sparse_text[0:2]

In [ ]:
tfidf = vectorizer.fit_transform(simple_text)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
tfidf.transpose()

In [ ]:
print(vectorizer.vocabulary_)